# Mini-Projet Spark : Analyse Gapminder
## Séances 1 et 2

# Séance 1 - Chargement et Pré-traitement

## Étape 1 - Définition de la structure de données

Dataset Gapminder : Données socio-économiques mondiales (1952-2007)

In [1]:
// Case class pour représenter une observation Gapminder
case class GapminderRecord(
  country: String,
  continent: String,
  year: Int,
  lifeExp: Double,
  pop: Long,
  gdpPercap: Double
)

## Étape 2 - Chargement des données avec SparkContext

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.{SparkConf, SparkContext}

// Initialisation Spark (si pas déjà fait)
val spark = SparkSession.builder()
  .appName("Gapminder Analysis")
  .enableHiveSupport()
  .getOrCreate()

val sc = spark.sparkContext

In [3]:
// Chargement du fichier CSV
val rawData = sc.textFile("https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/gapminder/gapminder.csv")

// Parsing des données (en ignorant l'en-tête)
val header = rawData.first()
val gapminderRDD = rawData
  .filter(row => row != header)  // Ignorer l'en-tête
  .map(line => {
    val fields = line.split(",")
    // Le CSV a une colonne index en position 0, on l'ignore
    GapminderRecord(
      country = fields(1).trim.replaceAll("\"", ""),
      continent = fields(2).trim.replaceAll("\"", ""),
      year = fields(3).toInt,
      lifeExp = fields(4).toDouble,
      pop = fields(5).toLong,
      gdpPercap = fields(6).toDouble
    )
  })

// Vérification du chargement
println(s"Nombre d'enregistrements chargés: ${gapminderRDD.count()}")
gapminderRDD.take(5).foreach(println)

## Persona utilisateur

**Dr. Sarah Chen**  
**Analyste en Développement International - ONU**

Objectifs :
- Identifier les pays avec les progressions les plus rapides en espérance de vie
- Analyser la corrélation entre PIB/habitant et espérance de vie par continent
- Détecter les inégalités de développement entre régions
- Préparer des rapports pour orienter l'aide au développement

# Séance 2 - DataFrame, Dimensions et SQL

## Étape 1 - Conversion RDD vers DataFrame

In [4]:
import spark.implicits._
import org.apache.spark.sql.functions._

// Conversion RDD vers DataFrame
val gapminderDF = gapminderRDD.toDF()

// Exploration du schéma et des données
gapminderDF.show(10, truncate = false)
gapminderDF.printSchema()

In [5]:
// Statistiques descriptives
gapminderDF.describe("lifeExp", "pop", "gdpPercap").show()

// Nombre total d'enregistrements
println(s"Nombre total d'observations: ${gapminderDF.count()}")

// Vérification des valeurs uniques
println(s"Nombre de pays: ${gapminderDF.select("country").distinct().count()}")
println(s"Nombre d'années: ${gapminderDF.select("year").distinct().count()}")
println(s"Continents: ")
gapminderDF.select("continent").distinct().show()

## Étape 2 - Extraction des dimensions

In [6]:
// Dimension Pays avec ID
val countryDim = gapminderDF
  .select("country", "continent")
  .distinct()
  .withColumn("country_id", monotonically_increasing_id())
  .select("country_id", "country", "continent")

countryDim.show(10)
println(s"Nombre de pays dans la dimension: ${countryDim.count()}")

In [7]:
// Dimension Temps
val timeDim = gapminderDF
  .select("year")
  .distinct()
  .withColumn("year_id", monotonically_increasing_id())
  .withColumn("decade", floor($"year" / 10) * 10)
  .select("year_id", "year", "decade")

timeDim.show()
println(s"Nombre d'années dans la dimension: ${timeDim.count()}")

In [8]:
// Table de faits avec jointures
val factsWithCountryId = gapminderDF
  .join(countryDim, Seq("country", "continent"), "inner")
  .join(timeDim, Seq("year"), "inner")

// Nettoyage - garder seulement les colonnes nécessaires
val factsTable = factsWithCountryId
  .select("country_id", "year_id", "lifeExp", "pop", "gdpPercap")
  .withColumnRenamed("lifeExp", "life_expectancy")
  .withColumnRenamed("pop", "population")
  .withColumnRenamed("gdpPercap", "gdp_per_capita")

factsTable.show(10)
println(s"Nombre d'enregistrements dans la table de faits: ${factsTable.count()}")

## Étape 3 - Création des tables Hive

In [9]:
// Sauvegarde des dimensions et faits en tables Hive
countryDim.write.mode("overwrite").saveAsTable("dim_country")
timeDim.write.mode("overwrite").saveAsTable("dim_time")
factsTable.write.mode("overwrite").saveAsTable("fact_gapminder")

// Vérification des tables créées
spark.sql("SHOW TABLES").show()

## Étape 4 - Requêtes SQL

In [10]:
// Requête 1: Top 10 pays par espérance de vie en 2007
val query1 = spark.sql("""
  SELECT 
    c.country,
    c.continent,
    f.life_expectancy,
    f.gdp_per_capita
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  JOIN dim_time t ON f.year_id = t.year_id
  WHERE t.year = 2007
  ORDER BY f.life_expectancy DESC
  LIMIT 10
""")

println("Top 10 pays par espérance de vie en 2007:")
query1.show()

In [11]:
// Requête 2: Évolution de l'espérance de vie moyenne par continent
val query2 = spark.sql("""
  SELECT 
    c.continent,
    t.decade,
    ROUND(AVG(f.life_expectancy), 2) as avg_life_exp,
    ROUND(AVG(f.gdp_per_capita), 2) as avg_gdp_per_cap
  FROM fact_gapminder f
  JOIN dim_country c ON f.country_id = c.country_id
  JOIN dim_time t ON f.year_id = t.year_id
  GROUP BY c.continent, t.decade
  ORDER BY c.continent, t.decade
""")

println("Évolution de l'espérance de vie moyenne par continent et décennie:")
query2.show(50)

In [12]:
// Requête 3: Pays avec la plus forte croissance d'espérance de vie
val query3 = spark.sql("""
  WITH life_exp_changes AS (
    SELECT 
      c.country,
      c.continent,
      MAX(CASE WHEN t.year = 2007 THEN f.life_expectancy END) as life_exp_2007,
      MAX(CASE WHEN t.year = 1952 THEN f.life_expectancy END) as life_exp_1952
    FROM fact_gapminder f
    JOIN dim_country c ON f.country_id = c.country_id
    JOIN dim_time t ON f.year_id = t.year_id
    WHERE t.year IN (1952, 2007)
    GROUP BY c.country, c.continent
  )
  SELECT 
    country,
    continent,
    ROUND(life_exp_1952, 2) as life_exp_1952,
    ROUND(life_exp_2007, 2) as life_exp_2007,
    ROUND(life_exp_2007 - life_exp_1952, 2) as improvement
  FROM life_exp_changes
  WHERE life_exp_1952 IS NOT NULL AND life_exp_2007 IS NOT NULL
  ORDER BY improvement DESC
  LIMIT 15
""")

println("Pays avec la plus forte croissance d'espérance de vie (1952-2007):")
query3.show()